In [5]:
import cv2
import mediapipe as mp
import numpy as np

In [6]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

## ANGLES

In [7]:
def calculate_angle(a,b,c):
    a = np.array(a) # first
    b = np.array(b) # mid
    c = np.array(c) # end
    
    rad = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(rad*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
    return angle

In [9]:
def get_coords():
    
    return args*2

In [33]:
#mp_holistic.PoseLandmark
lm_pose[getattr(mp_holistic.PoseLandmark, 'NOSE').value].x

0.5079775452613831

In [42]:
rshoulder = [lm_pose[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].x, 
             lm_pose[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].y]
rshoulder

[0.2555074095726013, 0.8443956971168518]

In [43]:
relbow = [lm_pose[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].x,
          lm_pose[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].y]
relbow

[0.15605904161930084, 1.225329041481018]

In [44]:
rwrist = [lm_pose[mp_holistic.PoseLandmark.__getattribute__(mp.ho).value].x,
          lm_pose[mp_holistic.PoseLandmark.RIGHT_WRIST.value].y]
rwrist

[0.16216664016246796, 1.5132442712783813]

In [45]:
calculate_angle(rshoulder, relbow, rwrist)

164.15339816928392

In [10]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(  # create holistic object
        min_detection_confidence=0.8,
        min_tracking_confidence=0.8) as holistic:
    while cap.isOpened():
        success, frame = cap.read()  # read video capture
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            break
        
        # Recolor Image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection
        results = holistic.process(image)
        
        # Recolor back to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image.flags.writeable = True
        
        # extract landmarks
        try:
            lm_pose = results.pose_landmarks.landmark
            # get coordinates
            rshoulder = [lm_pose[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].x, 
                         lm_pose[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].y]
            relbow = [lm_pose[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].x,
                      lm_pose[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].y]
            rwrist = [lm_pose[mp_holistic.PoseLandmark.RIGHT_WRIST.value].x,
                      lm_pose[mp_holistic.PoseLandmark.RIGHT_WRIST.value].y]
            # calculate angle
            angle = calculate_angle(rshoulder, relbow, rwrist)
            
            # visualize
            cv2.putText(image, str(round(angle,2)), 
                        tuple(np.multiply(relbow, [640,480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA
                        )
        except:
            pass
        try:
            lm_lhand = results.left_hand_landamarks.landmark
        except:
            pass
        try:
            lm_rhand = results.right_hand_landmarks.landmark
        except:
            pass

        # face
        """
        mp_drawing.draw_landmarks(
            image,
            results.face_landmarks,
            mp_holistic.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_contours_style())
        """
        # pose
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles
                .get_default_pose_landmarks_style())
        # Left hand
        mp_drawing.draw_landmarks(
            image,
            results.left_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles
                .get_default_hand_landmarks_style())
        # Right hand
        mp_drawing.draw_landmarks(
            image,
            results.right_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles
                .get_default_hand_landmarks_style())

        cv2.imshow('MediaPipe Holistic', image)
        if cv2.waitKey(1) == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()